# 3rd part: Image and texts, with EasyCLEF

In [10]:
import os
import ParserEasyClef
import ParserCACM
import modeles
import numpy as np
import query
import evaluation
# Auto reload the imported modules when running cells
%load_ext autoreload 
%autoreload 2

# Constants:
srcFolder = "easyCLEF08/" 
# File that contains all documents
srcFile = "easyCLEF08_text.txt"
# File that contains all relevances for queries
relevantsFile = "easyCLEF08_gt.txt"
# File that contains all queries
qryFile = "easyCLEF08_query.txt"

easyClef_txt = os.path.join(srcFolder, srcFile)
easyClef_qry = os.path.join(srcFolder, qryFile)
easyClef_rel = os.path.join(srcFolder, relevantsFile)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [9]:
import indexation 
import TextRepresenter

parser = ParserEasyClef.ParserEasyClef()
stemmer =  TextRepresenter.PorterStemmer()
idx = indexation.InMemoryIndex(easyClef_txt, 
                               parser, 
                               stemmer)
print("All doc ids:")
all_docs_id = idx.getDocsID()
print(all_docs_id)

All doc ids:
['15764', '24745', '7430', '23533', '31625', '8762', '30831', '40190', '39057', '4642', '13672', '39190', '22145', '18492', '39235', '39847', '39300', '6924', '35624', '40655', '19385', '20022', '12571', '27691', '37371', '37713', '10657', '18733', '31884', '26555', '31294', '15919', '6457', '35833', '39695', '2592', '9019', '13930', '20514', '4296', '31185', '37072', '21225', '9151', '37297', '27681', '3334', '25214', '39687', '35807', '27322', '37751', '7951', '39640', '40170', '30774', '27515', '13126', '11947', '16919', '30456', '9625', '14451', '10436', '32248', '17836', '21218', '37529', '7532', '19615', '40194', '30827', '31389', '31487', '7383', '40658', '30584', '4451', '16925', '1522', '20292', '31219', '35878', '16411', '8890', '19114', '4815', '27511', '30642', '25664', '38264', '19211', '1837', '30919', '10715', '11410', '19311', '30822', '32864', '7823', '38137', '39514', '3975', '2283', '11108', '13713', '16886', '3759', '8852', '9295', '8608', '3269', '4066

## Test baseline for some queries

In [23]:
modele_vect = modeles.Vectoriel(idx, modeles.TfidfWeighter(idx))
print("\n###### Testing QueryParserCACM: ###### ")
qp = query.QueryParserCACM(easyClef_qry, easyClef_rel)
qry = qp.nextQuery()
wantedId = [5, 3]
print("Searching for query", wantedId)
while qry is not None:
    if int(qry.getID()) in wantedId:
        print("Query")
        print(qry)
        dict_scores = modele_vect.getScores(stemmer.getTextRepresentation(qry.getText()))
        list_scores = [(docId, score) for docId, score in dict_scores.items()]
        irlist = evaluation.IRList(qry, list_scores)
        eval_precAtN = evaluation.PrecisionNDocuments(irlist)
        eval_CRAtN = evaluation.ClusterRecallNDocuments(irlist)
        n = 20
        print("Precision at %d docs:" % n)
        print(eval_precAtN.eval(n, verbose=True))
        print("Cluster Recall at %d docs:" % n)
        print(eval_CRAtN.eval(n, verbose=True))

        print(20 * '-')
    qry = qp.nextQuery()
    
print("Done.")



###### Testing QueryParserCACM: ###### 
Searching for query [5, 3]
Query
Query {id=3, txt='religious statue in the foreground. Relevant images will show a statue of one (or more) religious figures such as gods, angels, prophets etc. from any kind of religion in the foreground. Non-religious statues like war memorials or monuments are not relevant. Images with statues that are not the focus of the image (like the front view of church with many small statues) are not relevant. The statues of Easter Island are not relevant as they do not have any religious background.
', relevances=[28, 3147, 4531, 4533, 4759, 4815, 4895, 4896, 4900, 6501, 10657, 10999, 14520, 15330, 15973, 16819, 16993, 19135, 19194, 19198, 19211, 19259, 20356, 20357, 22777, 23108, 25966, 30011, 35678, 35780, 35782, 35783, 36035, 36036]}
Precision at 20 docs:
Result:  3407
Result:  3414
Result:  4815
Relevant, found docs = 1
Result:  38217
Result:  38216
Result:  38218
Result:  38221
Result:  38219
Result:  38222
Result